In [1]:
import math
import pandas

In [2]:
def calculate_entropy(feature):
    entropy = 0
    categories = feature.unique()
    total_count = feature.shape[0]
    for category in categories:
        cat_count =  feature[feature == category].count()
        entropy += cat_count / total_count * math.log(cat_count / total_count, 2) if cat_count > 0 else 0
    return -1 * entropy

In [3]:
def calculate_entropy_per_feature(df, feature_name, target_class):
    categories = df[feature_name].unique()
    entropies = {}
    
    for i, category in enumerate(categories):
        current_entropy = calculate_entropy(df[df[feature_name] == categories[i]][target_class])
        entropies[category] = current_entropy
        
    return entropies

In [4]:
def print_feature_entropy(entropies):
    for entropy in entropies:
        print("Entropy({} = {})\t=\t{}".format(feature, entropy, round(entropies[entropy], 4)))

In [5]:
def calculate_information_gain(overall_entropy, feature, entropy_per_feature):
    local_entropy = 0
    for category in entropy_per_feature:
        local_entropy += (feature[feature == category].shape[0] / feature.shape[0]) \
                    * entropy_per_feature[category]
    return overall_entropy - local_entropy

# Question 1

In [7]:
df1 = pandas.read_csv('tuto2_table1.txt', sep=" ")

print(df1)

    Name    Hair   Height    Build Lotion     Result
1  Sarah  blonde  average    light     no  sunburned
2   Dana  blonde     tall  average    yes       none
3   Alex   brown    short  average    yes       none
4  Annie  blonde    short  average     no  sunburned
5  Emily     red  average    heavy     no  sunburned
6   Pete   brown     tall    heavy     no       none
7   John   brown  average    heavy     no       none
8  Katie   brown    short    light    yes       none


## A/ Dataset entropy

In [8]:
q1_overall_entropy = calculate_entropy(df1['Result'])

print("Question:\t What is the entropy of this dataset with respect to the target classlabel Result?")
print("Answer\t\t", round(q1_overall_entropy, 4))

Question:	 What is the entropy of this dataset with respect to the target classlabel Result?
Answer		 0.9544


## B/ Decision tree & feature selection

### Step 1: Calculate overall dataset entropy (cf Q1/A/)

### Step 2: Calculate entropy for each feature

In [9]:
q1_list_features = list(df1.columns.values)
q1_list_features.remove("Name")
q1_list_features.remove("Result")

q1_target_class = 'Result'

q1_entropy_per_feature = {}

for feature in q1_list_features:
    curr_entropy = calculate_entropy_per_feature(df1, feature, q1_target_class)
    q1_entropy_per_feature[feature] = curr_entropy
    print("Feature:", feature)
    print()
    print_feature_entropy(curr_entropy)
    print()
    print()

Feature: Hair

Entropy(Hair = red)	=	-0.0
Entropy(Hair = blonde)	=	0.9183
Entropy(Hair = brown)	=	-0.0


Feature: Height

Entropy(Height = average)	=	0.9183
Entropy(Height = tall)	=	-0.0
Entropy(Height = short)	=	0.9183


Feature: Build

Entropy(Build = light)	=	1.0
Entropy(Build = average)	=	0.9183
Entropy(Build = heavy)	=	0.9183


Feature: Lotion

Entropy(Lotion = no)	=	0.971
Entropy(Lotion = yes)	=	-0.0




### Step 3: Calculate Information Gain for each feature

In [10]:
q1_information_gain_per_feature = {}

for feature in q1_list_features:
    curr_information_gain = calculate_information_gain(q1_overall_entropy, 
                                                       df1[feature],
                                                       q1_entropy_per_feature[feature])
    q1_information_gain_per_feature[feature] = curr_information_gain
    print("Feature:\t\t", feature)
    print("Information gain:\t", round(curr_information_gain, 4))
    print()

Feature:		 Hair
Information gain:	 0.6101

Feature:		 Height
Information gain:	 0.2657

Feature:		 Build
Information gain:	 0.0157

Feature:		 Lotion
Information gain:	 0.3476



In [11]:
print("Question:\nConstruct the decision tree that would be built with Information Gain for this dataset. Show your work for selection of the root feature in your tree.")

print()

print("Answer:\n\"Hair\" will be selected as it is the feature with the highest IG value. It perfectly classifies the data for Hair=brown & Hair=red. It will be used to split the root node of the tree. The case for Hair=blonde contains (2 sunburned, 1 none). We Can split these into pure child nodes using feature \"Lotion\".")

Question:
Construct the decision tree that would be built with Information Gain for this dataset. Show your work for selection of the root feature in your tree.

Answer:
"Hair" will be selected as it is the feature with the highest IG value. It perfectly classifies the data for Hair=brown & Hair=red. It will be used to split the root node of the tree. The case for Hair=blonde contains (2 sunburned, 1 none). We Can split these into pure child nodes using feature "Lotion".


## C/ New element classification

In [12]:
# hard-coded tree
def decision_tree(hair_color, applied_lotion):
    if hair_color == "Blonde":
        if applied_lotion:
            return False
        return True
    return False

new_person = {"Name": "Dana", 
              "Hair": "Blond", 
              "Height": "tall", 
              "Build":"average", 
              "Lotion":True}

for data in new_person:
    print("{}: {}".format(data, new_person[data]))

Name: Dana
Hair: Blond
Lotion: True
Build: average
Height: tall


In [13]:
print("Question:\tUsing your decision tree from (b), how would you classify the following example X?")
sunburned = decision_tree(new_person['Hair'], new_person['Lotion'])
if sunburned:   
    print("Answer:\t\t{}".format("sunburned"))
else:
    print("Answer:\t\t{}".format("not sunburned"))

Question:	Using your decision tree from (b), how would you classify the following example X?
Answer:		not sunburned


# Question 2

In [14]:
df2 = pandas.read_csv("tuto2_table2.txt", sep=" ")

print(df2)

    Credit  History  Debt  Income    Risk
0        1      bad   low   0to30    high
1        2      bad  high  30to60    high
2        3      bad   low   0to30    high
3        4  unknown  high  30to60    high
4        5  unknown  high   0to30    high
5        6     good  high   0to30    high
6        7      bad   low  over60  medium
7        8  unknown   low  30to60  medium
8        9     good  high  30to60  medium
9       10  unknown   low  over60     low
10      11  unknown   low  over60     low
11      12     good   low  over60     low
12      13     good  high  over60     low
13      14     good  high  over60     low


## A/ Dataset entropy

In [15]:
print("Question:\nWhat is the entropy of this dataset with respect to the target class label Risk based on the 14 examples above? ")

q2_overall_entropy = calculate_entropy(df2['Risk'])

print()

print("Answer:\n{}".format(round(q2_overall_entropy, 4)))

Question:
What is the entropy of this dataset with respect to the target class label Risk based on the 14 examples above? 

Answer:
1.5306


## B/ Entropy per feature

In [16]:
q2_list_features = list(df2.columns.values)
q2_list_features.remove("Credit")
q2_list_features.remove("Risk")

q2_target_class = "Risk"

q2_entropy_per_feature = {}

for feature in q2_list_features:
    curr_entropy = calculate_entropy_per_feature(df2, feature, q2_target_class)
    q2_entropy_per_feature[feature] = curr_entropy
    print_feature_entropy(curr_entropy)
    print()

Entropy(History = bad)	=	0.8113
Entropy(History = unknown)	=	1.5219
Entropy(History = good)	=	1.371

Entropy(Debt = high)	=	1.3788
Entropy(Debt = low)	=	1.5567

Entropy(Income = over60)	=	0.65
Entropy(Income = 30to60)	=	1.0
Entropy(Income = 0to30)	=	-0.0



## C/ Information gain and feature selection

In [17]:
q2_information_gain_per_feature = {}

for feature in q2_list_features:
    curr_information_gain = calculate_information_gain(q2_overall_entropy, 
                                                       df2[feature], 
                                                       q2_entropy_per_feature[feature])
    q2_information_gain_per_feature[feature] = curr_information_gain
    print("Feature:\t\t", feature)
    print("Information gain:\t", round(curr_information_gain, 4))
    print()

Feature:		 History
Information gain:	 0.2657

Feature:		 Debt
Information gain:	 0.0629

Feature:		 Income
Information gain:	 0.9663



In [18]:
print("Question:\nWhich one of the descriptive features would be selected by ID3 at the root of a decision tree? Explain your answer. Show all the steps of the calculations.")

print()

print("Answer:\n\"Income\" will be selected as the feature to split as it has the highest IG value.")

Question:
Which one of the descriptive features would be selected by ID3 at the root of a decision tree? Explain your answer. Show all the steps of the calculations.

Answer:
"Income" will be selected as the feature to split as it has the highest IG value.


## D/ Reflections on information gain criterions

> "Although information gain is usually a good measure for deciding the relevance of an attribute, it is not perfect. A notable problem occurs when information gain is applied to attributes that can take on a large number of distinct values. For example, suppose that one is building a decision tree for some data describing the customers of a business. Information gain is often used to decide which of the attributes are the most relevant, so they can be tested near the root of the tree. One of the input attributes might be the customer's credit card number. This attribute has a high mutual information, because it uniquely identifies each customer, but we do not want to include it in the decision tree: deciding how to treat a customer based on their credit card number is unlikely to generalise to customers we haven't seen before (overfitting)."


http://en.wikipedia.org/wiki/Information_gain_in_decision_trees